##RAG Pipeline - Data ingestion to vectorDB pipeline    

In [ ]:
# Bootstrap pip into the kernel if missing, then install required packages and run imports
import sys
import subprocess
import pkgutil

print("Notebook Python:", sys.executable)

# Ensure pip is available in this interpreter
if not pkgutil.find_loader("pip"):
    try:
        import ensurepip
        ensurepip.bootstrap(upgrade=True)
        print("ensurepip bootstrap completed")
    except Exception as e:
        print("ensurepip bootstrap failed:", e)

# Upgrade installer tools
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])

# Install required packages (no cache, prefer binary wheels)
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "--prefer-binary", "--no-cache-dir",
    "langchain-core>=1.0.0", "sentence-transformers", "chromadb", "scikit-learn"
])

# If PyTorch is needed and pip has trouble, install it separately using the official index (example shown as comment)
# subprocess.check_call([sys.executable, "-m", "pip", "install", "--index-url", "https://download.pytorch.org/whl/cpu", "torch==2.9.1", "--no-cache-dir", "--prefer-binary"])

# Direct imports (will raise if installation failed)
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_core.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

print("Packages installed and imports executed successfully.")


Notebook Python: c:\Users\kalpe\OneDrive\Desktop\RaGproject\.venv\Scripts\python.exe


C:\Users\kalpe\AppData\Local\Temp\ipykernel_14416\1571451199.py:9: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  if not pkgutil.find_loader("pip"):


ensurepip bootstrap completed


In [ ]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f" ✓ Loaded {len(documents)} pages")

        except Exception as e:  
            print(f" ✗ Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: Ai Easy Solutions 2019 Pattern.pdf
 ✓ Loaded 75 pages

Processing: LOI_Bhoi Kalpesh Sanjay.pdf
 ✓ Loaded 1 pages

Processing: zerotomastery.io+-+Complete+Ethical+Hacking+Bootcamp+Zero+to+Mastery+Guide.pdf
 ✓ Loaded 3 pages

Total documents loaded: 79
 ✓ Loaded 3 pages

Total documents loaded: 79


In [ ]:
all_pdf_documents

[Document(metadata={'producer': 'Adobe Scan for Android 22.05.19-regular', 'creator': 'Adobe Scan for Android 22.05.19-regular', 'creationdate': '', 'source': '..\\data\\pdf\\Ai Easy Solutions 2019 Pattern.pdf', 'total_pages': 75, 'page': 0, 'page_label': '1', 'source_file': 'Ai Easy Solutions 2019 Pattern.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'Adobe Scan for Android 22.05.19-regular', 'creator': 'Adobe Scan for Android 22.05.19-regular', 'creationdate': '', 'source': '..\\data\\pdf\\Ai Easy Solutions 2019 Pattern.pdf', 'total_pages': 75, 'page': 1, 'page_label': '2', 'source_file': 'Ai Easy Solutions 2019 Pattern.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'Adobe Scan for Android 22.05.19-regular', 'creator': 'Adobe Scan for Android 22.05.19-regular', 'creationdate': '', 'source': '..\\data\\pdf\\Ai Easy Solutions 2019 Pattern.pdf', 'total_pages': 75, 'page': 2, 'page_label': '3', 'source_file': 'Ai Easy Solutions

In [ ]:
## text splitting get into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split into {len(split_docs)} chunks")
    return split_docs

# Split the loaded PDF documents
    if split_docs:
        print(f"\nexample chunk:")
        print(f"content, {split_docs[0].page_content[:200]}... ")
        print(f"metadata: {split_docs[0].metadata}")
    return split_docs

In [ ]:
chunks= split_documents(all_pdf_documents)
chunks
#i just split data into chunks now 

Split into 10 chunks


##embedding and vector DB


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid # its generate unique ids
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


^C


ModuleNotFoundError: No module named 'sentence_transformers'